# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [256]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [257]:
driver = webdriver.Chrome()

In [258]:
driver.get('https://www.tdlr.texas.gov/tools_search/')

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [259]:
input_box = driver.find_element_by_name("mcrdata")

In [260]:
input_box.send_keys("006179570C")

In [261]:
button = driver.find_element_by_xpath('//*[@id="submit3"]')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

In [ ]:
contractors = driver.find_elements_by_tag_name('tr')

for contractor in contractors:

In [115]:
owner_name = contractors[5].find_elements_by_tag_name('td')[0].text

In [ ]:
cont_name = contractors[4].find_elements_by_tag_name('td')[0].text

In [116]:
phone_name = contractors[6].find_elements_by_tag_name('td')[0].text

In [117]:
status_status = contractors[7].find_elements_by_tag_name('td')[1].text

In [130]:
address_address = contractors[8].find_elements_by_tag_name('td')[1].text.split("\n")[-6:7]

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [263]:
def get_towtr_info(row):
    
    driver.get("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR Number'])

    contractors = driver.find_elements_by_tag_name('tr')
    
    for contractor in contractors:

        cont_name = contractors[4].find_elements_by_tag_name('td')[0].text
        owner_name = contractors[5].find_elements_by_tag_name('td')[0].text
        phone_name = contractors[6].find_elements_by_tag_name('td')[0].text
        status_status = contractors[7].find_elements_by_tag_name('td')[1].text
        address_address = contractors[8].find_elements_by_tag_name('td')[1].text.split("\n")[-6:7]

    return pd.Series({
        'name': cont_name,
        'owner': owner_name,
        'status': status_status,
        'address': address_address,
        'phone': phone_name
        
    })

### Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [264]:
import pandas as pd

df1 = pd.read_csv("trucks-subset.csv")
df1.head()

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


In [185]:
driver = webdriver.Chrome()

In [265]:
new_df = df1.apply(get_towtr_info, axis=1).join(df1)
new_df.head()

,name,owner,status,address,phone,TDLR Number
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Status: Active,"[1617 COUNTY ROAD 4130, BONHAM, TX. 75418]",Phone: 9032276464,006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Status: Expired,"[13619 BRETT JACKSON RD, FORT WORTH, TX. 76179]",Phone: 8173330706,006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",Phone: 8066544404,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [266]:
def add_url(row):

    contractors = driver.find_elements_by_tag_name('tr')
    
    for contractor in contractors:

        cont_name = contractors[4].find_elements_by_tag_name('td')[0].text
        owner_name = contractors[5].find_elements_by_tag_name('td')[0].text
        phone_name = contractors[6].find_elements_by_tag_name('td')[0].text
        status_status = contractors[7].find_elements_by_tag_name('td')[1].text
        address_address = contractors[8].find_elements_by_tag_name('td')[1].text.split("\n")[-6:7]
        url = str("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR Number'])
    
    return pd.Series({
        'name1': cont_name,
        'owner1': owner_name,
        'status1': status_status,
        'address1': address_address,
        'phone1': phone_name,
        'url1': url})

In [267]:
new_df_plus_url = df.apply(add_url, axis=1).join(new_df)
new_df_plus_url.head()

,name1,owner1,status1,address1,phone1,url1,name,owner,status,address,phone,TDLR Number
0,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",Phone: 8066544404,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Status: Active,"[1617 COUNTY ROAD 4130, BONHAM, TX. 75418]",Phone: 9032276464,006507931C
1,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",Phone: 8066544404,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Status: Expired,"[13619 BRETT JACKSON RD, FORT WORTH, TX. 76179]",Phone: 8173330706,006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",Phone: 8066544404,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"[4501 W CEMETERY RD, CANYON, TX. 79015]",Phone: 8066544404,006502097C


# Go through each row of the dataset, printing out information about each tow truck company.

#Now will be scraping inside of your function.

#The business name
Owner/operator
Phone number
License status (Active, Expired, Etc)
Physical address
Just print it out for now.

TIP: use .apply
TIP: You'll be using the code you wrote before, but converted into a function
TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!
TIP: Make sure you adjust any variables so you don't scrape the same page again and again

In [247]:
def  truck_company_info(row): 
    
    contractors = driver.find_elements_by_tag_name('tr')
    
    for contractor in contractors:

        cont_name = contractors[4].find_elements_by_tag_name('td')[0].text
        owner_name = contractors[5].find_elements_by_tag_name('td')[0].text
        phone_name = contractors[6].find_elements_by_tag_name('td')[0].text
        status_status = contractors[7].find_elements_by_tag_name('td')[1].text
        address_address = contractors[8].find_elements_by_tag_name('td')[1].text.split("\n")[-6:7]
        url = str("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR Number'])
    
    return cont_name, owner_name, phone_name, status_status, address_address, url

In [248]:
new_df_plus_url.apply(truck_company_info, axis=1)

0    (Name:   BARRY MICHAEL SMITH, Owner/Officer:   BARRY MICHAEL SMITH / OWNER, Phone:   8066544404, Status:  Active, [4501 W CEMETERY RD, CANYON, TX. 79015], https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C)
1    (Name:   BARRY MICHAEL SMITH, Owner/Officer:   BARRY MICHAEL SMITH / OWNER, Phone:   8066544404, Status:  Active, [4501 W CEMETERY RD, CANYON, TX. 79015], https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C)
2    (Name:   BARRY MICHAEL SMITH, Owner/Officer:   BARRY MICHAEL SMITH / OWNER, Phone:   8066544404, Status:  Active, [4501 W CEMETERY RD, CANYON, TX. 79015], https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C)
dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [268]:
def  truck_company_info_extended(row): 
    
    contractors = driver.find_elements_by_tag_name('tr')
    
    for contractor in contractors:

        cont_name = contractors[4].find_elements_by_tag_name('td')[0].text
        owner_name = contractors[5].find_elements_by_tag_name('td')[0].text
        phone_name = contractors[6].find_elements_by_tag_name('td')[0].text
        status_status = contractors[7].find_elements_by_tag_name('td')[1].text
        address_address = contractors[8].find_elements_by_tag_name('td')[1].text.split("\n")[-6:7]
        url = str("https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=" + row['TDLR Number'])
    
    return pd.Series({
        'name1': cont_name,
        'owner1': owner_name,
        'status1': status_status,
        'address1': address_address,
        'phone1': phone_name,
        'url1': url})

In [269]:
tow_trucks_extended=new_df_plus_url.apply(truck_company_info_extended, axis=1)

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [270]:
tow_trucks_extended.to_csv("tow_trucs_extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [271]:
pd.read_csv("tow_trucs_extended.csv")

,name1,owner1,status1,address1,phone1,url1
0,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"['4501 W CEMETERY RD', 'CANYON, TX. 79015']",Phone: 8066544404,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"['4501 W CEMETERY RD', 'CANYON, TX. 79015']",Phone: 8066544404,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Status: Active,"['4501 W CEMETERY RD', 'CANYON, TX. 79015']",Phone: 8066544404,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**